In [1]:
HopsworksProjectName = "TaxiDemandPrediction"

In [2]:
import os
import sys
sys.path.append("../")

import APIKey

HopsworksAPIKey = APIKey.HOPSWORKSAPIKEY

# Fetching Historical Data from 2022 Onwards from the TLC Trip Data 

In [7]:
from datetime import datetime 
import pandas as pd
sys.path.append("../src/")
import data

StartYear = 2022
EndYear = datetime.now().year
print(f"Downloading Data from Year {StartYear} to Year {EndYear}")

Rides = pd.DataFrame()

for y in range(StartYear, EndYear+1):
    
    #Download one whole Year worth of Data
    RidesOneWholeYear = data.LoadRawData(y)
    
    #Append Rows
    Rides = pd.concat([Rides, RidesOneWholeYear])

File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
File 2023-06 was already in local storage
File 2023-07 is not available
File 2023-08 is not available
File 2023-09 is not available
File 2023-10 is not available
File 2023-11 is not available
File 2023-12 is not available


In [10]:
TS_Data = data.TransformRawDataIntoTSData(Rides)

100%|█████████████████████████████████████████| 262/262 [00:05<00:00, 49.36it/s]


In [32]:
TS_Data.rename(columns = {"PickupHour":"pickup_hour", "NumOfRides":"numrides", "PickupLocationID":"pickup_location_id"}, inplace = True)

In [33]:
import hopsworks

In [34]:
Project = hopsworks.login(project = HopsworksProjectName, api_key_value = HopsworksAPIKey)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/104605


In [35]:
FeatureStore = Project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [39]:
FeatureGroupName = "ts_hourly_featuregroup"
FeatureGroupVersion = 1

In [40]:
FeatureGroup = FeatureStore.get_or_create_feature_group(name = FeatureGroupName,
                                                        version = FeatureGroupVersion,
                                                        description = "TimeSeries Data at Hourly Frequency",
                                                        primary_key = ["pickup_location_id", "pickup_hour"],
                                                        event_time = "pickup_hour"
                                                       )

In [41]:
FeatureGroup.insert(TS_Data, write_options = {"wait_for_job":False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/104605/fs/104524/fg/109065


Uploading Dataframe: 100.00% |█| Rows 3433248/3433248 | Elapsed Time: 03:46 | Re


Launching job: ts_hourly_featuregroup_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/104605/jobs/named/ts_hourly_featuregroup_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f84e4b72350>, None)